In [1]:
!pip install transformers

In [2]:
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer
import torch
from PIL import Image



#To Encode or Decode Model


In [3]:
model= VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

Some weights of the model checkpoint at nlpconnect/vit-gpt2-image-captioning were not used when initializing VisionEncoderDecoderModel: ['decoder.transformer.h.7.attn.masked_bias', 'decoder.transformer.h.2.crossattention.masked_bias', 'decoder.transformer.h.6.attn.masked_bias', 'decoder.transformer.h.3.crossattention.bias', 'decoder.transformer.h.3.attn.masked_bias', 'decoder.transformer.h.3.crossattention.masked_bias', 'decoder.transformer.h.2.attn.masked_bias', 'decoder.transformer.h.5.crossattention.bias', 'decoder.transformer.h.5.crossattention.masked_bias', 'decoder.transformer.h.2.crossattention.bias', 'decoder.transformer.h.9.attn.masked_bias', 'decoder.transformer.h.7.crossattention.masked_bias', 'decoder.transformer.h.7.crossattention.bias', 'decoder.transformer.h.8.crossattention.masked_bias', 'decoder.transformer.h.4.attn.masked_bias', 'decoder.transformer.h.10.crossattention.bias', 'decoder.transformer.h.10.attn.bias', 'decoder.transformer.h.4.attn.bias', 'decoder.transform

#To Extract Features from Pretrained Model

In [4]:
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
tokenizer=AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_featur

# Image Captioning Function 

In [6]:
max_length = 16
num_beams = 4

gen_kwargs = {"max_length":max_length, "num_beams": num_beams}


In [7]:
def predict_step(images):

  pixel_values=feature_extractor(images=images, return_tensors="pt").pixel_values
  pixel_values=pixel_values.to(device)

  output_ids=model.generate(pixel_values, **gen_kwargs)

  preds=tokenizer.batch_decode(output_ids, skip_special_tokens=True)
  preds=[pred.strip() for pred in preds]

  return preds


In [ ]:
predict_step(["/content/Image1.png"])

['a man kicking a soccer ball on a field']

# Interface

In [8]:
!pip install gradio

In [9]:
import gradio as gr

In [ ]:
inputs=[
    gr.inputs.Image(type="pil", label="ORiginal Image")
]

outputs=[
    gr.outputs.Textbox(label="Caption")
]

title="Image Captioning"
description="AI based Caption generator"
article = "<a href = 'https://huggingface.co/nlpconnect/vit-gpt2-image-captioning'>Model Repo hugging face model hub</a>"
examples = [
    ["Image1.png"]
]

gr.Interface(
    predict_step,
    inputs,
    outputs,
    title=title,
    description=description,
    article=article,
    examples=examples,
    theme="huggingFace"

).launch(debug=True, enable_queue=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://68a06abc7ccb49818c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
